In [5]:
from aif360.metrics.classification_metric import ClassificationMetric
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings

from utils import make_dataset, log_fairness_metrics

warnings.filterwarnings('ignore')


TARGET_COLS = 'two_year_recid'
COMPAS_SCORES_COLS = 'decile_score'
NUMERICAL_FEATURE_COLS = ['age',
                          'juv_fel_count','juv_misd_count','juv_other_count',
                          'priors_count','jail_time']
CATEGORICAL_FEATURE_COLS = ['sex','race',
                            'c_charge_degree']
FEATURE_NAMES = NUMERICAL_FEATURE_COLS+CATEGORICAL_FEATURE_COLS

PROTECTED_COLS = ['sex','race']

BIAS_INFO = {'favorable_label':0,
             'unfavorable_label':1,
             'protected_columns':['race'],
            }

data = pd.read_csv('../data/processed/compas-scores-two-years-processed.csv')
data.head()

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,jail_time,sex,race,c_charge_degree,decile_score,two_year_recid,compas_score
0,69,0,0,0,0,0.0,1,1,1,1,0,0.1
1,34,0,0,0,0,10.0,1,2,1,3,1,0.3
2,24,0,0,1,4,1.0,1,2,1,4,1,0.4
3,23,0,1,0,1,0.0,1,2,1,8,0,0.8
4,43,0,0,0,2,0.0,1,1,1,1,0,0.1


In [6]:
train, test = train_test_split(data, test_size=0.2, random_state=1234)

# Prejudice Remover

In [7]:
from aif360.algorithms.inprocessing import PrejudiceRemover


PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 2},
                                          {'race': 1},
                                          {'race': 4},
                                          {'race': 5},
                                          {'race': 6}],
                   'privileged_groups':[{'race': 3}]
                  }

ground_truth_train = make_dataset(train[FEATURE_NAMES+[TARGET_COLS]], 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
ground_truth_test = make_dataset(test[FEATURE_NAMES+[TARGET_COLS]], 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)

clf = PrejudiceRemover(sensitive_attr='race')

clf.fit(ground_truth_train)
prediction_test = clf.predict(ground_truth_test)

# Problem with AIF360 implementation
prediction_test.labels = (prediction_test.labels==0).astype(int)

roc_auc = roc_auc_score(test[TARGET_COLS], prediction_test.labels)
clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)

In [8]:
import neptune
from neptunecontrib.api.utils import get_filepaths

neptune.init('jakub-czakon/model-fairness')
neptune.create_experiment(name='baseline',tags=['inprocessing','race','prejudice-remover'], 
                          upload_source_files=get_filepaths(extensions=['.py','.ipynb']))

neptune.log_metric('roc_auc',roc_auc)
log_fairness_metrics(clf_metric)

neptune.stop()

https://ui.neptune.ml/jakub-czakon/model-fairness/e/MOD-87


W0912 17:59:40.860211 139675374794496 experiments.py:366] Invalid metric value: nan for channel false_positive_rate_ratio. Metrics with nan or +/-inf values will not be sent to server
W0912 17:59:40.862397 139675374794496 experiments.py:366] Invalid metric value: inf for channel false_negative_rate_ratio. Metrics with nan or +/-inf values will not be sent to server
W0912 17:59:40.864208 139675374794496 experiments.py:366] Invalid metric value: inf for channel false_omission_rate_ratio. Metrics with nan or +/-inf values will not be sent to server
W0912 17:59:40.865956 139675374794496 experiments.py:366] Invalid metric value: nan for channel false_discovery_rate_ratio. Metrics with nan or +/-inf values will not be sent to server
W0912 17:59:40.868365 139675374794496 experiments.py:366] Invalid metric value: inf for channel error_rate_ratio. Metrics with nan or +/-inf values will not be sent to server


# Adversarial Fairness

In [6]:
from sklearn.preprocessing import MinMaxScaler

def collapse_group(x):
    if x!=3:
        return 0
    else:
        return 1

train['race'] = train['race'].apply(collapse_group)
test['race'] = test['race'].apply(collapse_group)

PRIVILEGED_INFO = {'unprivileged_groups':[{'race': 0}],
                   'privileged_groups':[{'race': 1}]
                  }

scaler = MinMaxScaler()
train[FEATURE_NAMES] = scaler.fit_transform(train[FEATURE_NAMES])
test[FEATURE_NAMES] = scaler.transform(test[FEATURE_NAMES])

In [7]:
# sess.close()
# tf.reset_default_graph()

In [8]:
from aif360.algorithms.inprocessing import AdversarialDebiasing
import tensorflow as tf

ground_truth_train = make_dataset(train[FEATURE_NAMES+[TARGET_COLS]], 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)
ground_truth_test = make_dataset(test[FEATURE_NAMES+[TARGET_COLS]], 'two_year_recid', **BIAS_INFO, **PRIVILEGED_INFO)

sess = tf.Session()

model_params = {'classifier_num_hidden_units':200, 
                'num_epochs':50,
                'batch_size':128,
                'adversary_loss_weight':0.1,
                'debias':True
               }

clf = AdversarialDebiasing(scope_name='debiased_classifier', sess=sess, seed=1234, 
                           **PRIVILEGED_INFO,
                           **model_params
                           )

clf.fit(ground_truth_train)
prediction_test = clf.predict(ground_truth_test)

roc_auc = roc_auc_score(test[TARGET_COLS], prediction_test.labels)
clf_metric = ClassificationMetric(ground_truth_test, prediction_test,**PRIVILEGED_INFO)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




epoch 0; iter: 0; batch classifier loss: 0.698464; batch adversarial loss: 0.669505
epoch 1; iter: 0; batch classifier loss: 0.666032; batch adversarial loss: 0.691279
epoch 2; iter: 0; batch classifier loss: 0.649043; batch adversarial loss: 0.683533
epoch 3; iter: 0; batch classifier loss: 0.675367; batch adversarial loss: 0.671310
epoch 4; iter: 0; batch classifier loss: 0.594715; batch adversarial loss: 0.659164


In [9]:
import neptune
from neptunecontrib.api.utils import get_filepaths

neptune.init('jakub-czakon/model-fairness')
neptune.create_experiment(name='baseline',
                          params=model_params,
                          tags=['inprocessing','race','adversarial-debiasing'], 
                          upload_source_files=get_filepaths(extensions=['.py','.ipynb']))

neptune.log_metric('roc_auc',roc_auc)
log_fairness_metrics(clf_metric)

neptune.stop()

https://ui.neptune.ml/jakub-czakon/model-fairness/e/MOD-74
